In [169]:
import pandas as pd
import os
from datetime import datetime
import utils.data_utils as data_utils
from collections import defaultdict

LONG_TERM_ACTIONS = ['walking', 'eating', 'smoking', 'discussion']
ACTIONS = data_utils.define_actions('all')
ACTIONS.sort(key=len)

In [170]:
ACTIONS

['eating',
 'posing',
 'walking',
 'smoking',
 'phoning',
 'sitting',
 'waiting',
 'greeting',
 'purchases',
 'discussion',
 'directions',
 'walkingdog',
 'sittingdown',
 'takingphoto',
 'walkingtogether']

In [171]:
def csv_iter():
    for f in os.listdir('./checkpoint/test'):
        if not f.startswith('short_term'):
            continue
        try:
            datetime.strptime(f[-23:-4], "%d-%m-%Y-%H:%M:%S")
            if f.split('.')[-1] == 'csv':
                day = int(f[-23:-21])
                if day > 20:
                    yield f
        except:
            pass

def extend_df(df, long_term=True):
    if long_term:
        actions = LONG_TERM_ACTIONS
        time_lens = [560, 1000]
    else:
        actions = ACTIONS
        time_lens = [80, 160, 320, 400]
    columns_mapping = defaultdict(list)
    for time_len in time_lens:
        for action_name in actions:
            columns_mapping['3d' + str(time_len)].append(action_name + '3d' + str(time_len))
#             columns_mapping['usingfulltestset_3d' + str(time_len)].append('usingfulltestset_' + action_name + '3d' + str(time_len))

    for key in columns_mapping:
        df[key] = df[columns_mapping[key]].mean(axis=1)
    
    return list(columns_mapping.keys())

def get_summary_df(merged):
    cols = list(set([i[:-5] for i in merged.columns]))
    new_cols = []
    for col in cols:
        col_name = [c for c in merged.columns if c.startswith(col)]
        new_cols.append(col+'_mean')
        new_cols.append(col+'_std')
        merged[col+'_mean'] = merged[col_name].mean(axis=1)
        merged[col+'_std'] = merged[col_name].std(axis=1)
    return merged[new_cols]

In [172]:
# df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    print(i, f)
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='v_3d')
#     df_lst.append(df[extend_df(df, long_term=True)].add_suffix('_run' + str(i)))
    extend_df(df, long_term=False)
    cols_to_keep = [i for i in df.columns if i not in ['epoch', 'lr', 't_l', 'v_3d'] and not i.startswith('usingfulltestset')]
    
    cols_to_keep = [col for col in cols_to_keep if col.split('3')[0] in ACTIONS and int(col.split('3d')[1]) in [80, 160, 320, 400] or col.startswith('3')]
    
    mini_df_lst.append(df[cols_to_keep].head(1).add_suffix('_run' + str(i)).reset_index(drop=True))

merged = pd.concat(mini_df_lst, axis=1)

0 short_termmain_3d_3D_in10_out10_dct_n_30_28-06-2020-17:11:11.csv
1 short_termmain_3d_3D_in10_out10_dct_n_30_26-06-2020-17:51:18.csv
2 short_termmain_3d_3D_in10_out10_dct_n_30_27-06-2020-05:36:39.csv
3 short_termmain_3d_3D_in10_out10_dct_n_30_27-06-2020-17:23:34.csv
4 short_termmain_3d_3D_in10_out10_dct_n_30_28-06-2020-05:20:17.csv


In [173]:
merged

,walking3d80_run0,walking3d160_run0,walking3d320_run0,walking3d400_run0,eating3d80_run0,eating3d160_run0,eating3d320_run0,eating3d400_run0,smoking3d80_run0,smoking3d160_run0,...,walkingdog3d320_run4,walkingdog3d400_run4,walkingtogether3d80_run4,walkingtogether3d160_run4,walkingtogether3d320_run4,walkingtogether3d400_run4,3d80_run4,3d160_run4,3d320_run4,3d400_run4
0,9.225759,15.620593,29.53867,34.02269,8.293664,18.330667,39.044098,48.458885,6.709925,13.287291,...,96.397354,115.334625,8.893956,18.114157,35.208073,44.087776,11.500675,24.413511,50.369607,60.890268


In [174]:
ACTIONS

['eating',
 'posing',
 'walking',
 'smoking',
 'phoning',
 'sitting',
 'waiting',
 'greeting',
 'purchases',
 'discussion',
 'directions',
 'walkingdog',
 'sittingdown',
 'takingphoto',
 'walkingtogether']

In [175]:
my_cols = []
ORDERED_ACTIONS = ['walking', 'eating', 'smoking', 'discussion', 'directions', 'greeting', 'phoning', 'posing', 'purchases', 'sitting',
                  'sittingdown', 'takingphoto', 'waiting', 'walkingdog', 'walkingtogether']

assert(len(set(ACTIONS).intersection(set(ORDERED_ACTIONS))) == len(ACTIONS))
for name in ORDERED_ACTIONS:
    my_cols.append(name + '3d80_mean')
    my_cols.append(name + '3d160_mean')
    my_cols.append(name + '3d320_mean')
    my_cols.append(name + '3d400_mean')
my_cols.append('3d80_mean')
my_cols.append('3d160_mean')
my_cols.append('3d320_mean')
my_cols.append('3d400_mean')

In [176]:
tmp = get_summary_df(merged)
tmp[my_cols]

,walking3d80_mean,walking3d160_mean,walking3d320_mean,walking3d400_mean,eating3d80_mean,eating3d160_mean,eating3d320_mean,eating3d400_mean,smoking3d80_mean,smoking3d160_mean,...,walkingdog3d320_mean,walkingdog3d400_mean,walkingtogether3d80_mean,walkingtogether3d160_mean,walkingtogether3d320_mean,walkingtogether3d400_mean,3d80_mean,3d160_mean,3d320_mean,3d400_mean
0,9.316807,15.985963,30.10947,34.133907,8.475722,18.59529,38.116001,46.625439,6.964761,13.836292,...,99.68416,119.486273,8.91467,18.690227,35.530943,44.348322,11.416297,24.375663,50.417804,60.950869


In [187]:
values = [str(i).split('.')[0] + '.' + str(i).split('.')[1][:1] for i in list(tmp[my_cols].iloc[0])]
'&'.join(values[16+6*4:])

'11.3&28.0&54.8&64.8&6.4&15.6&41.4&53.5&9.2&21.7&55.9&72.1&29.3&56.4&99.6&119.4&8.9&18.6&35.5&44.3&11.4&24.3&50.4&60.9'

In [63]:
# mini_df_lst[0]

In [64]:
# mini_df_lst[1]

In [65]:
# mini_df_lst[2]

In [66]:
# mini_df_lst[3]

In [67]:
# mini_df_lst[4]

In [167]:
with open('shorttermcopy.txt', 'r') as f:
    counter = 0
    for i, line in enumerate(f):
        if len(line) > 5:
            tmp = line.split(']')[-1]
            tmp = tmp.strip('\n')
            tmp = tmp.strip(' ')
            print('&'.join(tmp.split(' ')))
            counter += 1
            if counter == 3:
                print()
                print()
                counter = 0

23.8&40.4&62.9&70.9&17.6&34.7&71.9&87.7&19.7&36.6&61.8&73.9&31.7&61.3&96.0&103.5
17.1&31.2&53.8&61.5&13.7&25.9&52.5&63.3&11.1&21.0&33.4&38.3&18.9&39.3&67.7&75.7
8.9&15.7&29.2&33.4&8.8&18.9&39.4&47.2&7.8&14.9&25.3&28.7&9.8&22.1&39.6&44.1


36.5&56.4&81.5&97.3&37.9&74.1&139.0&158.8&25.6&44.4&74.0&84.2&27.9&54.7&131.3&160.8&40.8&71.8&104.2&109.8&34.5&69.9&126.3&141.6
22.0&37.2&59.6&73.4&24.5&46.2&90.0&103.1&17.2&29.7&53.4&61.3&16.1&35.6&86.2&105.6&29.4&54.9&82.2&93.0&19.8&42.4&77.0&88.4
12.6&24.4&48.2&58.4&14.5&30.5&74.2&89.0&11.5&20.2&37.9&43.2&9.4&23.9&66.2&82.9&19.6&38.5&64.4&72.2&10.7&24.6&50.6&62.0


28.6&55.3&101.6&118.9&23.6&47.4&94.0&112.7&29.5&60.5&119.9&140.6&60.5&101.9&160.8&188.3&23.5&45.0&71.3&82.8&30.8&57.0&99.8&115.5
17.1&34.9&66.3&77.7&14.0&27.2&53.8&66.2&17.9&36.5&74.9&90.7&40.6&74.7&116.6&138.7&15.0&29.9&54.3&65.8&19.6&37.8&68.1&80.2
11.4&27.6&56.4&67.6&6.8&15.2&38.2&49.6&9.5&22.0&57.5&73.9&32.2&58.0&102.2&122.7&8.9&18.4&35.3&44.3&12.1&25.0&51.0&61.3




In [147]:
a = 5

In [148]:
a

5